# Form.io AI Workflow (Abstracted Guide)
This notebook documents an abstract workflow you can rebuild in your own app:

1. Accept a human-readable specification of a form (text or file).
2. Call an LLM (default: OpenAI `gpt-5`) with a strict system prompt to produce Form.io JSON.
3. Parse and normalize the model output to guaranteed JSON.
4. Optionally apply a secondary pass to merge a datamodel.
5. Render/preview the JSON (here via a lightweight HTML preview using the Form.io CDN).

Swap the provider call to use Anthropic or Mistral if you prefer.


In [ ]:
import os, json, requests

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  # set in your environment
MODEL = 'gpt-5'  # default

SYSTEM = (
    "You are a Form.io form designer. Return only valid JSON.\n"
    "Requirements: The object should start with 'display' then 'components'.\n"
    "No prose, no markdown fences. Keep labels concise."
)

def call_openai(spec_text: str) -> dict:
    url = 'https://api.openai.com/v1/chat/completions'
    headers = { 'Authorization': f'Bearer {OPENAI_API_KEY}', 'Content-Type': 'application/json' }
    body = {
        'model': MODEL,
        'messages': [
            { 'role': 'system', 'content': SYSTEM },
            { 'role': 'user', 'content': f'Specification:\n\n{spec_text}' }
        ]
    }
    resp = requests.post(url, headers=headers, json=body)
    resp.raise_for_status()
    content = resp.json()['choices'][0]['message']['content']
    # try direct JSON or fenced JSON
    try:
        return json.loads(content)
    except Exception:
        import re
        m = re.search(r"```json\n([\s\S]*?)```", content) or re.search(r"```([\s\S]*?)```", content)
        if m:
            return json.loads(m.group(1))
        raise

spec = 'Create a signup form for car insurance applicants'
generated = call_openai(spec)
# Normalize minimal structure
if isinstance(generated, list):
    generated = { 'display': 'form', 'components': generated }
generated.setdefault('display', 'form')
generated.setdefault('components', [])
print(json.dumps(generated, indent=2))


In [ ]:
# Optional second pass: merge a datamodel by prompting again
DATAMODEL = 'name,email,phone\n'  # example CSV-like text
SYSTEM_MOD = ("You are a Form.io form designer. Modify the provided JSON to wire fields per the datamodel. "
              "No prose; return valid JSON with 'display' then 'components'.")

def apply_datamodel(current_json: dict, datamodel_text: str) -> dict:
    url = 'https://api.openai.com/v1/chat/completions'
    headers = { 'Authorization': f'Bearer {OPENAI_API_KEY}', 'Content-Type': 'application/json' }
    body = {
        'model': MODEL,
        'messages': [
            { 'role': 'system', 'content': SYSTEM_MOD },
            { 'role': 'user', 'content': f'Current JSON:\n{json.dumps(current_json)}\n\nDatamodel:\n{datamodel_text}' }
        ]
    }
    resp = requests.post(url, headers=headers, json=body)
    resp.raise_for_status()
    content = resp.json()['choices'][0]['message']['content']
    try:
        return json.loads(content)
    except Exception:
        import re
        m = re.search(r"```json\n([\s\S]*?)```", content) or re.search(r"```([\s\S]*?)```", content)
        if m:
            return json.loads(m.group(1))
        raise

updated = apply_datamodel(generated, DATAMODEL)
print(json.dumps(updated, indent=2))


In [ ]:
# Minimal HTML preview using Form.io CDN (works in many notebook environments)
from IPython.display import HTML
html = f'''
<!doctype html>
<html><head>
  <link rel="stylesheet" href="https://cdn.form.io/js/formio.full.min.css"/>
</head><body>
  <div id="formio" style="padding:12px"></div>
  <script src="https://cdn.form.io/js/formio.full.min.js"></script>
  <script>
    const schema = {json.dumps(generated)};
    Formio.createForm(document.getElementById('formio'), schema);
  </script>
</body></html>
'''
HTML(html)
